# Импорты

In [26]:
from transformers import AutoImageProcessor, SwinForImageClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoFeatureExtractor
import torch
# import torchvision
import evaluate
import numpy as np
from datasets import load_dataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


[codecarbon INFO @ 01:07:11] Energy consumed for RAM : 0.000248 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 01:07:11] Energy consumed for all GPUs : 0.002397 kWh. Total GPU Power : 13.308 W
[codecarbon INFO @ 01:07:11] Energy consumed for all CPUs : 0.001980 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 01:07:11] 0.004625 kWh of electricity used since the beginning.


# Dataset

In [27]:
dataset = load_dataset("mvkvc/artifact-100k") # subset of ArtiFact labled with "ai" and "real" tags
# dataset = load_dataset("imagefolder", data_dir="c:/Users/bes-s/OneDrive/Документы/NN/datasets/artifacts")
dataset = dataset.with_format(type="torch", device=device)

batch_size = 20

# small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))

model_name= 'test_trainer7'
# model_name = 'microsoft/swin-large-patch4-window7-224'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['image']], return_tensors='pt')
    inputs['label'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

[codecarbon INFO @ 01:07:16] Energy consumed for RAM : 0.000124 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 01:07:16] Energy consumed for all GPUs : 0.001259 kWh. Total GPU Power : 41.929 W
[codecarbon INFO @ 01:07:16] Energy consumed for all CPUs : 0.000990 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 01:07:16] 0.002372 kWh of electricity used since the beginning.
C:\Users\bes-s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
C:\Users\bes-s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead

# Model Creation

In [28]:
metric = evaluate.load("accuracy")
def compute_metrics(p):
  # function which calculates accuracy for a certain set of predictions
  return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

def collate_fn(batch):
  #data collator
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

Номерация моделей


4. base
5. large
6. base artifacts
7. large artifacts
8. base 100k -> artifacts
9. large 100k -> artifacts
10. base artifacts -> 100k
11. large artifacts -> 100k отменен! Причина: 97 часов на обучение

In [29]:
labels = dataset['train'].features['label'].names

model = SwinForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes = True,
)

training_args = TrainingArguments(
    f"test_trainer11",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10, #Эпохи
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)



[codecarbon INFO @ 01:07:26] Energy consumed for RAM : 0.000273 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 01:07:26] Energy consumed for all GPUs : 0.002454 kWh. Total GPU Power : 13.803 W
[codecarbon INFO @ 01:07:26] Energy consumed for all CPUs : 0.002178 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 01:07:26] 0.004905 kWh of electricity used since the beginning.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

[codecarbon INFO @ 01:07:28] [setup] RAM Tracking...
[codecarbon INFO @ 01:07:28] [setup] GPU Tracking...
[codecarbon INFO @ 01:07:28] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 01:07:28] [setup] CPU Tracking...
[codecarbon WARNING @ 01:07:28] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 01:07:30] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-8600K CPU @ 3.60GHz
[codecarbon INFO @ 01:07:30] >>> Tracker's metadata:
[codecarbon INFO @ 01:07:30]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 01:07:30]   Python version: 3.9.13
[codecarbon INFO @ 01:07:30]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 01:07:30]   Available RAM : 15.866 GB
[codecarbon INFO @ 01:07:30]   CPU count: 6
[codecarbon INFO @ 01:07:30]   CPU model: Intel(R) Core(TM) i5-8600K CPU @ 3.60GHz
[codecarbon INFO @ 01:07:30]   GPU count: 1
[codecarbon INFO @ 01:07:30]   GPU model: 1 x NVIDIA GeForce RTX 4070 Ti


[codecarbon INFO @ 01:07:31] Energy consumed for RAM : 0.000149 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 01:07:31] Energy consumed for all GPUs : 0.001308 kWh. Total GPU Power : 11.832 W
[codecarbon INFO @ 01:07:31] Energy consumed for all CPUs : 0.001188 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 01:07:31] 0.002645 kWh of electricity used since the beginning.


# TRAIN IT

In [31]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

# Evaluate on validation set


  0%|          | 0/11250 [00:00<?, ?it/s]

[codecarbon INFO @ 01:07:41] Energy consumed for RAM : 0.000297 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 01:07:41] Energy consumed for all GPUs : 0.002874 kWh. Total GPU Power : 100.816 W
[codecarbon INFO @ 01:07:41] Energy consumed for all CPUs : 0.002376 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 01:07:41] 0.005548 kWh of electricity used since the beginning.
[codecarbon INFO @ 01:07:46] Energy consumed for RAM : 0.000173 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 01:07:46] Energy consumed for all GPUs : 0.001560 kWh. Total GPU Power : 60.514 W
[codecarbon INFO @ 01:07:46] Energy consumed for all CPUs : 0.001386 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 01:07:46] 0.003119 kWh of electricity used since the beginning.
[codecarbon INFO @ 01:07:49] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 01:07:49] Energy consumed for all GPUs : 0.000267 kWh. Total GPU Power : 63.865 W
[codecarbon INFO @ 01:07:4

In [16]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

  0%|          | 0/55 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =     5.9421
  eval_accuracy           =     0.6447
  eval_loss               =      0.636
  eval_runtime            = 0:00:13.77
  eval_samples_per_second =     79.497
  eval_steps_per_second   =      3.993
